In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name ='RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' #'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
env = gym.make(env_name)
print(env.action_space.low)
print(env.action_space.high)

[-1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-05-12 09:01:53,804	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-12_09-01-53_13520/logs.
2019-05-12 09:01:53,915	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:50348 to respond...
2019-05-12 09:01:54,037	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:40934 to respond...
2019-05-12 09:01:54,044	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-05-12 09:01:54,102	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-12_09-01-53_13520/logs.
2019-05-12 09:01:54,104	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:50348',
 'object_store_address': '/tmp/ray/session_2019-05-12_09-01-53_13520/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-12_09-01-53_13520/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks

a2c_config = {
    'GAMMA' : 0.99,
    'TAU' : 0.9,
    'LEARNING_RATE' : 1e-4,
    'NAME' : 'Supermario',
    'SCORE_TO_WIN' : 5000,
    'EPISODES_TO_LOG' : 20, 
    'LIVES_REWARD' : 5,
    'GRAD_NORM' : 0.5,
    'ENTROPY_COEF' : 0.000,
    'TRUNCATE_GRADS' : True,
    'ENV_NAME' : env_name,
    'PPO' : True,
    'E_CLIP' : 0.2,
    'NUM_ACTORS' : 16,
    'STEPS_NUM' : 128,
    'MINIBATCH_SIZE' : 512,
    'MINI_EPOCHS' : 4,
    'CRITIC_COEF' : 1,
}
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
action_space = env.action_space

agent = A2CAgent(sess,'mario', env.observation_space, False, action_space, a2c_config)
#agent.restore('nn/a2cBipedalWalker-v2')
#agent.train()



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


2019-05-12 09:01:58,319	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


Instructions for updating:
Use tf.cast instead.


In [6]:
ray.shutdown()

In [7]:
import numpy as np
agent.restore('nn/a2cRoboschoolAnt-v1')

def evaluate(env,t_max=5000):
    rewards = []
    env._max_episode_steps = 5000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(t_max):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        #print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/a2cRoboschoolAnt-v1
reset
reward: 10112.89573068732


In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()